In [1]:
import requests
from collections import defaultdict
from datetime import datetime


class FamagaClient:
    def __init__(self, api_key, session_id):
        self.base_url = "https://api.famaga.org"
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Cookie": f"PHPSESSID={session_id}"
        }

    def list_offers_by_deal_id(self, deal_id):
        """List offers by deal ID"""
        response = requests.get(f"{self.base_url}/requisition?request={deal_id}", headers=self.headers)
        return response.json()

    def parts_selected_at_offer(self, offer_id):
        """Fetch parts selected at a specific offer"""
        response = requests.get(f"{self.base_url}/requisition/{offer_id}/products", headers=self.headers)
        return response.json()

    def list_offers_by_client_id(self, client_id):
        """Retrieve offers by client ID"""
        response = requests.get(f"{self.base_url}/requisition?client={client_id}", headers=self.headers)
        return response.json()

    def list_current_offer_details(self, offer_id):
        """List details of the current offer in a specific format"""
        parts = self.parts_selected_at_offer(offer_id)
        formatted_parts = []
        for part in parts:
            formatted_part = f"({part['articul']} {part['brand_title']}) margin: {part['margin_ru']}%, sell: {part['price_end']}$ qty. {part['count']}"
            formatted_parts.append(formatted_part)
        return "\n".join(formatted_parts)

    def find_parts_client_bought_before(self, client_id, current_offer_parts):
        """Return the parts that a client has bought before"""
        all_offers = self.list_offers_by_client_id(client_id)
        past_parts = []
        for offer in all_offers.get('content', []):
            parts = self.parts_selected_at_offer(offer['request']['id'])
            past_parts.extend(parts)

        bought_before = []
        current_parts_articul = [part['articul'] for part in current_offer_parts]
        for part in past_parts:
            if part['articul'] in current_parts_articul:
                bought_before.append(part)

        return bought_before

    def get_client_purchase_history_formatted(self, client_id, exluded_deal: int = None):
        """Generate formatted purchase history for a client"""
        history_data = self.list_offers_by_client_id(client_id)
        purchases = defaultdict(list)

        for item in history_data.get('content', []):
            date_key = datetime.strptime(item['created'], '%Y-%m-%dT%H:%M:%S%z').date()
            request_id = int(item['request']['id'])
            if exluded_deal is None or request_id != exluded_deal:
                parts = self.parts_selected_at_offer(request_id)

                for part in parts:
                    line = f"{item['id']} ({part['articul']} {part['brand_title']}) margin: {part['margin_ru']}%, sell: {part['price_end']}$ qty. {part['count']}, request id {part['request_id']}"
                    purchases[date_key].append(line)

        # Formatting the result
        result = "[CLIENT PURCHASE HISTORY]\n**Client purchase history:**\n"
        for date in sorted(purchases):
            result += f"Date: {date}\n"
            result += "\n".join(purchases[date]) + "\n\n"
        result += "[/CLIENT PURCHASE HISTORY]"
        return result

In [2]:
api_key = "YXBpZmFtYWdhcnU6RHpJVFd1Lk1COUV4LjNmdERsZ01YYlcvb0VFcW9NLw"
session_id = "085qpt4eflu39a0dg7hjhr5mdu"
famaga = FamagaClient(api_key, session_id)

In [19]:
resp = famaga.list_offers_by_deal_id(476119)
resp

{'content': [{'id': 359513,
   'request': {'id': 476119,
    'firm': {'id': 115706,
     'title': 'EUROSANAT',
     'type': '2',
     'conversion': '',
     'link': 'index.php?option=com_oscrm&view=firmform&id=115706&Itemid=219'},
    'initials': 'AKA',
    'brand': {'id': 12766,
     'title': 'LOOS International / Bosch Industriekessel',
     'conversion': '12.9',
     'link': 'index.php?option=com_oscrm&view=brandform&id=12766&Itemid=219'},
    'contractor': {'id': 32450,
     'title': 'C.P.Biro d.o.o.',
     'link': 'index.php?option=com_oscrm&view=contractorform&id=32450&Itemid=216',
     'created': '2022-03-14T07:17:23+00:00',
     'modified': '2024-03-13T12:56:16+00:00'},
    'curator': {'id': 884,
     'name': 'Maria Pasca',
     'username': 'Maria Pasca',
     'email': 'maria.pasca@famaga.de'},
    'requestManagers': [[]],
    'cost': '21231.26'},
   'created': '2024-02-06T09:24:10+00:00',
   'lastsend': '2024-02-06T13:02:31+00:00',
   'processing_time': '',
   'decision': 6,
 

In [26]:
from pydantic import BaseModel, HttpUrl, EmailStr
from typing import List, Optional
from datetime import datetime
import json

class Firm(BaseModel):
    id: int
    title: str
    type: str
    conversion: Optional[str]
    link: str

class Brand(BaseModel):
    id: int
    title: str
    conversion: Optional[str]
    link: str

class Contractor(BaseModel):
    id: int
    title: str
    link: str
    created: datetime
    modified: str

class Curator(BaseModel):
    id: int
    name: str
    username: str
    email: EmailStr

class Request(BaseModel):
    id: int
    firm: Firm
    initials: str
    brand: Brand
    contractor: Contractor
    curator: Curator
    requestManagers: List[List]
    cost: str

class Content(BaseModel):
    id: int
    request: Request
    created: datetime
    lastsend: datetime
    processing_time: Optional[str]
    decision: int
    link: str
    target_price: int
    target_price_solved: bool
    state: bool
    proform: Optional[dict]

class Response(BaseModel):
    content: List[Content]
    total: int
    limit: int
    page: int


In [11]:
mr = Response.model_validate(resp)

In [20]:
mr.content[0].request.firm.id

115706

In [28]:
import csv


resp = famaga.list_offers_by_deal_id(467721)
mr = Response.model_validate(resp)

offers_resp = famaga.list_offers_by_client_id(mr.content[0].request.firm.id)

In [29]:
content_list = offers_resp['content']

# Specify the CSV file path
csv_file_path = 'offers.csv'

# Write to CSV file
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the header
    header = ['id', 'request_id', 'firm_id', 'firm_title', 'initials', 'brand_id', 'brand_title', 'contractor_id', 'contractor_title', 'curator_name', 'curator_email', 'cost', 'created', 'lastsend', 'processing_time', 'decision', 'link', 'target_price', 'target_price_solved', 'state']
    writer.writerow(header)

    # Write the data
    for item in content_list:
        row = [
            item['id'],
            item['request']['id'],
            item['request']['firm']['id'],
            item['request']['firm']['title'],
            item['request']['initials'],
            item['request']['brand']['id'],
            item['request']['brand']['title'],
            item['request']['contractor']['id'],
            item['request']['contractor']['title'],
            item['request']['curator']['name'],
            item['request']['curator']['email'],
            item['request']['cost'],
            item['created'],
            item['lastsend'],
            item['processing_time'],
            item['decision'],
            item['link'],
            item['target_price'],
            item['target_price_solved'],
            item['state']
        ]
        writer.writerow(row)

print(f"CSV file created at {csv_file_path}")

CSV file created at offers.csv


In [ ]:
mss = {
    'Endress+Hauser': ['E+H'],
    '': ['sefsef', 'sdfs']
}

In [ ]:
70234324